<a href="https://colab.research.google.com/github/ShoaibMuhammad123/Pytorch/blob/main/Pytorch11_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt


# **fashion-mnist_test.csv**

In [ ]:
df = pd.read_csv('/content/fashion-mnist_test.csv')

In [ ]:
df.shape

(10000, 785)

In [ ]:
df.head(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,...,pixel735,pixel736,pixel737,pixel738,pixel739,pixel740,pixel741,pixel742,pixel743,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,44,88,99,122,123,80,0,0,0,0,...,0,212,240,213,239,233,239,231,232,236,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,235,210,241,222,171,220,199,236,27,0,...,0,0,0,0,0,220,244,206,0,87,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,2,0,0,0,0,0,140,116,85,109,...,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0


In [ ]:
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 3, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y

array([0, 1, 2, ..., 8, 8, 1])

In [ ]:
torch.manual_seed(42)

In [ ]:
# Check for GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# devide

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,random_state=42)

In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
x_train[:4]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
x_test[:4]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# **Custom Dataset**

In [ ]:
class CustomDataset(Dataset):

  def __init__(self,features,labels):
    super().__init__()

    self.features = torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index],self.labels[index]



In [ ]:
train_data = CustomDataset(x_train,y_train)
test_data = CustomDataset(x_test,y_test)


In [ ]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True,pin_memory=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False,pin_memory=True)

# **Model Architecture**

In [ ]:
class CNN(nn.Module):
  def __init__(self,input_features  ):
    super().__init__()

    # feature extraction part of CNN
    self.features = nn.Sequential(
      nn.Conv2d(input_features,32,kernel_size=3,padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.MaxPool2d(kernel_size=2,stride=2),


      nn.Conv2d(32,64,kernel_size=3,padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(kernel_size=2,stride=2)


    )
    # classifier part of CNN

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.ReLU(),
        nn.Dropout(p=0.4),  # the dropout we have applied latter for only overfitting

        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(64,10)
    )

  def forward(self,x):
    x = self.features(x)
    x = self.classifier(x)

    return x

In [ ]:
learning_rate =0.1
epochs = 8

In [ ]:
x_train.shape[0]

8000

In [ ]:
model = CNN(1)

# if cpu then comment it else not
# model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-4)

In [ ]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features , batch_labels in train_loader:

    # if you can run in gpu then comment out this
    # batch_features , batch_labels = batch_features.to(device) , batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs,batch_labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss +loss.item()

  avg_loss = total_epoch_loss/ len(train_loader)

  print(f'Epoch {epoch+1} loss {avg_loss}')

Epoch 1 loss 0.43302336207032205
Epoch 2 loss 0.41292572689056395
Epoch 3 loss 0.39269804432988165
Epoch 4 loss 0.37101738291978836
Epoch 5 loss 0.3479085935652256
Epoch 6 loss 0.32915009865164757
Epoch 7 loss 0.32421719935536386
Epoch 8 loss 0.30544643810391425


# **Evaluation on Test data**

In [ ]:
correct =0
total =0

with torch.no_grad():
  for epoch in range(epochs):

    for batch_features,batch_labels in test_loader:

      outputs = model(batch_features)

      _,predicted = torch.max(outputs,1)
      total+=batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()
print(correct/total)






0.851625
